# Profile Photo Verification

In [ ]:
!python3 -m pip install --upgrade pip
!pip install opencv-python
!pip install tensorflow
!pip install matplotlib

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

## Use OpenPose FACE/COCO/MPI to find points on the body that specify the pose

In [ ]:
params = {"face": {"proto": 'pose/face/pose_deploy_linevec.prototxt', "model": 'pose/face/pose_iter_116000.caffemodel', "points": 18},
            "coco": {"proto": 'pose/coco/pose_deploy_linevec.prototxt', "model": 'pose/coco/pose_iter_440000.caffemodel', "points": 18},
            "mpi": {"proto": 'pose/mpi/pose_deploy_linevec_faster_4_stages.prototxt', "model": 'pose/mpi/pose_iter_160000.caffemodel', "points": 15}}

def detect_points(img, model, threshold=0):
    (imgheight, imgwidth) = img.shape[:2]
    inp = cv2.dnn.blobFromImage(img, 1.0 / 255, (imgheight, imgwidth), (0, 0, 0), swapRB=False, crop=False)

    # load OpenPose
    net = cv2.dnn.readNetFromCaffe(params[model]["proto"], params[model]["model"])
    net.setInput(inp)
    out = net.forward()

    H, W = out.shape[2:]

    # Extract the pose data points, keep the ones where we are above threshold confidence that they are correct
    points = []
    points_kept = []
    for i in range(params[model]["points"]):
        probMap = out[0, i, :, :]
        minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)

        x = (imgwidth * point[0]) / W
        y = (imgheight * point[1]) / H

        if prob > threshold:
            pt = (int(x), int(y))
            points.append(pt)
            points_kept.append(i)
        else:
            points.append(None)
            
    return points, points_kept

## Check that the data points match

In [ ]:
MATCH = 1
NO_MATCH = 0

def check_for_match(img1_pts, img2_pts, max_deviation=1):
    # Transform the data points such that the points from both images are centered horizontally and vertically
    y_min1 = min([pt[1] for pt in img1_pts])
    y_min2 = min([pt[1] for pt in img2_pts])
    x_center1 = np.mean([pt[0] for pt in img1_pts])
    x_center2 = np.mean([pt[0] for pt in img2_pts])
    avg_center = np.mean([x_center1, x_center2])
    trans_img1_data = [(avg_center + pt[0] - x_center1, pt[1] - y_min1) for pt in img1_pts]
    trans_img2_data = [(avg_center + pt[0] - x_center2, pt[1] - y_min2) for pt in img2_pts]

    # Plot the datapoints
    fig, ax = plt.subplots(figsize=(3,3))
    ax.set_title("Data Points Superimposed")
    ax.scatter(*zip(*trans_img1_data), color='blue')
    ax.scatter(*zip(*trans_img2_data), color='orange')
    ax.set_ylim(ax.get_ylim()[::-1])
    fig.show()

    # Compute distances between each pair of data points
    dist = lambda pt1, pt2 : ((pt1[0] - pt2[0])**2 + (pt1[1] - pt2[1])**2)**(1/2)
    pt_distances = list(map(dist, trans_img1_data, trans_img2_data))
    #print(pt_distances)

    # Determine if there are any outliers
    mean = np.mean(pt_distances)
    std = np.std(pt_distances)
    #print(mean, std)

    if any(pt_distances < mean - std * max_deviation) or any(pt_distances > mean + std * max_deviation):
        return NO_MATCH

    return MATCH

# Example

In [ ]:
profile_img = cv2.imread('example/profile_photo.jpg')
profile_img_gray = cv2.cvtColor(profile_img, cv2.COLOR_BGR2RGB)

pose_img = cv2.imread('example/pose.jpg')
pose_img_gray = cv2.cvtColor(pose_img, cv2.COLOR_BGR2RGB)

verification_img = cv2.imread('example/verification_photo.jpg')
verification_img_gray = cv2.cvtColor(verification_img, cv2.COLOR_BGR2RGB)

bad_verification_img = cv2.imread('example/bad_verification_photo.jpg')
bad_verification_img_gray = cv2.cvtColor(verification_img, cv2.COLOR_BGR2RGB)

Try a pose that does match

In [ ]:
img1_data, img1_pts_kept = detect_points(pose_img, "mpi", threshold=0.5)
img2_data, img2_pts_kept = detect_points(verification_img, "mpi", threshold=0.5)
img3_data, img3_pts_kept = detect_points(profile_img, "face", threshold=0.5)
img4_data, img4_pts_kept = detect_points(verification_img, "face", threshold=0.5)

pts_to_keep = list(set.intersection(set(img1_pts_kept), set(img2_pts_kept)))
img1_data = list(np.array(img1_data)[pts_to_keep])
img2_data = list(np.array(img2_data)[pts_to_keep])
print(len(pts_to_keep))     # TODO: should force min on this

pts_to_keep = list(set.intersection(set(img3_pts_kept), set(img4_pts_kept)))
img3_data = list(np.array(img3_data)[pts_to_keep])
img4_data = list(np.array(img4_data)[pts_to_keep])
print(len(pts_to_keep))     # TODO: should force min on this

pose_match = check_for_match(img1_data, img2_data, max_deviation=3)
face_match = check_for_match(img3_data, img4_data, max_deviation=2)

if pose_match == MATCH and face_match == MATCH:
    print("Verified")
else:
    print("NOT Verified")

Try a pose that does not match

In [ ]:
img1_data, img1_pts_kept = detect_pose(pose_img, threshold=0.5)
img2_data, img2_pts_kept = detect_pose(bad_verification_img, threshold=0.5)

pts_to_keep = list(set.intersection(set(img1_pts_kept), set(img2_pts_kept)))
img1_data = list(np.array(img1_data)[pts_to_keep])
img2_data = list(np.array(img2_data)[pts_to_keep])
print(len(pts_to_keep))

pose_match = check_for_match(img1_data, img2_data, max_deviation=3)